In [1]:
import numpy as np
from tocotron2 import Tocotron2

In [2]:
test_text_data = np.random.randint(0, 1000, (100, 30))
test_audio_data = np.random.randint(0, 100, (100, 1000))





In [3]:
params_json = {
    "run_folder": "c:\\Users\\1\\Desktop\\PersonalFriendProject\\models_save\\Tacotron2_save",
    "params_path": "C:\\Users\\1\\Desktop\\TelegramAIBotProject\\bot\\models_params\\Tacotron2.json",
    "encoder_params": {

        "total_labels_n": 1000,
        "embedding_dim": 100,
        "max_sequence_lenght": 1000,

        "conv_params": {

            "filters": [32, 32, 128, 128],
            "kernel_size": [3, 3, 3, 3],
            "strides": [1, 2, 2, 1],
            "dropout_rates": 0.26,
            "activations": ["linear", "linear", "linear", "linear"]
        },
        "lstm_units": 215
    },

    "decoder_params": {
        "total_labels_n": 10000,
        "max_sequence_lenght": 10000,
        "embedding_dim": 100,
        "dense_params": {
            "units": 215,
            "dropout_rates": 0.26,
            "activation": "linear"
        },

        "lstm_params": {
            "units": 215,
            "dropout_rates": 0.26,
            "layers_n": 3
        },

        "conv_params": {
            "filters": [32, 32, 128, 215],
            "kernel_size": [3, 3, 3, 3],
            "strides": [1, 2, 2, 1],
            "dropout_rates": 0.26,
            "activations": ["linear", "linear", "linear", "linear"],
            "output_activation": "tanh"
        }
    },
    
    "wavenet_params": {
        "layers_n": 3,
        "filters": 215,
        "kernel_size": 5,
        "output_dim": 10000,
        "max_seq_len": 126
    }
}

In [4]:
tocotron_model = Tocotron2(params_json=params_json)

C:\Users\1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\layer.py:1295: UserWarning: Layer 'wave_net_layer' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Shapes used to initialize variables must be fully-defined (no `None` dimensions). Received: shape=(None, 10000) for variable path='wave_net_layer/dense/kernel'''
  warnings.warn(
C:\Users\1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\layer.py:361: UserWarning: `build()

In [5]:
tocotron_model.model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, None, 430) │    766,112 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ [(None, 1),       │  2,994,983 │ input_layer_1[0]… │
│ (Functional)        │ (None, None,      │            │ functional_1[0][… │
│                     │ 215)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ wave_net_layer      │ (None, 10000)     │    231,340 │ functional_3[0][… │
│ (WaveNetLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ wave_net_layer_1    │ (None, 10000)     │          0 │ wave_net_layer[0… │
│ (WaveNetLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10000)     │          0 │ wave_net_layer_1… │
│                     │                   │            │ wave_net_layer[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ wave_net_layer_2    │ (None, 10000)     │          0 │ add[0][0]         │
│ (WaveNetLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 10000)     │          0 │ wave_net_layer_2… │
│                     │                   │            │ add[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,992,435 (15.23 MB)

 Trainable params: 3,992,435 (15.23 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
encoder_output = tocotron_model.encoder.predict(test_text_data)
decoder_output = tocotron_model.decoder.predict([test_audio_data, encoder_output])
result_waves = tocotron_model.wavenet_model.predict(decoder_output[1])


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 615ms/step
3/4 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step  WARNING:tensorflow:6 out of the last 12 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002287F18BA60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step


In [7]:
print(encoder_output.shape, decoder_output[0].shape, decoder_output[1].shape, result_waves.shape)

(100, 8, 430) (100, 1) (100, 250, 215) (100, 126, 10000)


In [8]:
from datasets import load_dataset

ds = load_dataset("facebook/multilingual_librispeech", "german")

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/3469 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/469942 [00:00<?, ? examples/s]

Generating 9_hours split:   0%|          | 0/2194 [00:00<?, ? examples/s]

Generating 1_hours split:   0%|          | 0/241 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/62 [00:00<?, ?it/s]

In [26]:
audio_data = [ds["train"][sample_number]["audio"]["array"][:10000] for sample_number in range(1000) 
              if ds["train"][sample_number]["audio"]["array"].shape[0] >= 10000]
audio_data = np.asarray(audio_data)
print(audio_data.shape)

(1000, 10000)


In [31]:
train_texts = [ds["train"][sample_n]["transcript"] for sample_n in range(1000)]

In [33]:
tocotron_model.train_model(train_text_sequences=train_texts, train_voice_sequences=audio_data, epochs=100, batch_size=32)

ergeben in sein schicksal konnte man ihn nicht nennen aber dieser letzte schlag konnte ihn doch nicht in aufregung versetzen so schien es wenigstens
